# Hyper Parameter Tuning

- Parameter
    - model parameter : 회귀 계수(가중치), 절편 등 모델의 학습 '대상'이 되는 변수
    - hyper parameter : 모델 설정과 관련해 '직접 지정'할 수 있는 매개변수

In [1]:
from sklearn.datasets import load_iris

iris_input, iris_target = load_iris(return_X_y=True)

### GridSearchCV

- 하이퍼 파라미터의 값 목록이나 값의 범위를 제공하면 그 안에서 최적의 하이퍼파라미터 조합을 탐색

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()

params = {
    'n_neighbors' : range(1, 13, 2)
}

grid = GridSearchCV(knn, params, scoring='accuracy', cv=5)
grid.fit(iris_input, iris_target)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",KNeighborsClassifier()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'n_neighbors': range(1, 13, 2)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate pa

In [3]:
grid.best_params_

{'n_neighbors': 7}

In [4]:
grid.best_estimator_

,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",7
,"weights weights: {'uniform', 'distance'}, callable or None, default='uniform'Weight function used in prediction. Possible values:- 'uniform' : uniform weights. All points in each neighborhood are weighted equally.- 'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.- [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.Refer to the example entitled:ref:`sphx_glr_auto_examples_neighbors_plot_classification.py`showing the impact of the `weights` parameter on the decisionboundary.",'uniform'
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"p p: float, default=2Power parameter for the Minkowski metric. When p = 1, this is equivalentto using manhattan_distance (l1), and euclidean_distance (l2) for p = 2.For arbitrary p, minkowski_distance (l_p) is used. This parameter is expectedto be positive.",2
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'minkowski'
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.Doesn't affect :meth:`fit` method.",None


In [5]:
grid.best_score_

np.float64(0.9800000000000001)

In [6]:
knn = grid.best_estimator_
knn.score(iris_input, iris_target)

0.9733333333333334

### RandomSearchCV

- 하이퍼 파라미터의 값 목록이나 값의 범위를 제공하면, 그 안에서 랜덤하게 조합을 뽑아내 최적의 하이퍼 파라미터 조합을 탐색
    - 장점 : 탐색 범위가 넓을 때 짧은 시간 내 좋은 결과를 얻을 수 있음
    - 단점 : 랜덤하게 값을 추출해 계산하므로 전역 최적값을 놓칠 수 있음

In [7]:
from sklearn.model_selection import RandomizedSearchCV

knn = KNeighborsClassifier()

params = {
    'n_neighbors' : range(1, 100, 2)
}

rd_search = RandomizedSearchCV(knn, params, cv=5, n_iter=10, random_state=0)
rd_search.fit(iris_input, iris_target)

,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",KNeighborsClassifier()
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'n_neighbors': range(1, 100, 2)}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",10
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: in

In [8]:
print(rd_search.best_params_)
print(rd_search.best_estimator_)
print(rd_search.best_score_)

{'n_neighbors': 5}
KNeighborsClassifier()
0.9733333333333334


In [9]:
rd_search.cv_results_

{'mean_fit_time': array([0.        , 0.        , 0.        , 0.00024819, 0.        ,
        0.00020065, 0.        , 0.        , 0.        , 0.00030274]),
 'std_fit_time': array([0.        , 0.        , 0.        , 0.00049639, 0.        ,
        0.00040131, 0.        , 0.        , 0.        , 0.0004035 ]),
 'mean_score_time': array([1.91278458e-03, 0.00000000e+00, 3.27892303e-03, 4.62784815e-01,
        0.00000000e+00, 3.16514969e-03, 3.47743034e-03, 3.70798111e-03,
        0.00000000e+00, 3.98969650e-04]),
 'std_score_time': array([3.82556915e-03, 0.00000000e+00, 6.12972284e-03, 9.17920235e-01,
        0.00000000e+00, 6.15913580e-03, 6.95486069e-03, 6.77152756e-03,
        0.00000000e+00, 4.88672333e-04]),
 'param_n_neighbors': masked_array(data=[57, 23, 21, 83, 5, 55, 77, 63, 45, 9],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value=999999),
 'params': [{'n_neighbors': 57},
  {'n_neighbors': 23},
  {'n_n

### [한번 해보기] 생선 다중분류 with GridSearchCV

- 생선의 Weight, Length, Diagonal, Height, Width를 사용해 생선의 Species를 예측하는 문제를 LogisticRegression 모델로 해결해보세요.
- GridSearchCV를 활용해 최적의 성능을 내는 Hyper Parameter를 찾아보세요

In [10]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

fish = pd.read_csv('./data/fish.csv')
fish_input = fish[['Width', 'Length', 'Diagonal', 'Height', 'Width']]
fish_target = fish['Species']

lr = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(fish_input, fish_target, random_state=0)

lr.fit(X_train, y_train)
knn = KNeighborsClassifier()

params = {
    'n_neighbors' : range(1, 13, 2)
}

grid = GridSearchCV(knn, params, scoring='accuracy', cv=5)
grid.fit(iris_input, iris_target)

c:\Users\Playdata\miniconda3\envs\mlstudy_env\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",KNeighborsClassifier()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'n_neighbors': range(1, 13, 2)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate pa

### HyperOpt

In [11]:
from hyperopt import hp
import hyperopt

# 탐색 공간
search_space = {
    'x' : hp.quniform('x', -10, 10, 1)
    , 'y' : hp.quniform('y', -15, 15, 1)
}

# 목적 함수
def objective(search_space) :
    x = search_space['x']
    y = search_space['y']
    
    return {
        'loss' : x**2 + 20 * y
        , 'status' : hyperopt.STATUS_OK
    }

c:\Users\Playdata\miniconda3\envs\mlstudy_env\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [12]:
from hyperopt import fmin, tpe, Trials

# 탐색 과정을 저장하는 객체
trials = Trials()

# fmin() : 목적 함수의 최소값을 찾는 함수
best_val = fmin(
    fn=objective            # 목적 함수
    , space=search_space    # 검색 공간
    , algo=tpe.suggest      # 베이지안 최적화 적용
    , max_evals=500         # 반복 횟수 지정
    , trials=trials         # 탐생 과정 저장
)

best_val

100%|██████████| 500/500 [00:06<00:00, 80.06trial/s, best loss: -300.0] 


{'x': np.float64(0.0), 'y': np.float64(-15.0)}

In [13]:
# 시도별 목적함수 반환값
trials.results

[{'loss': 161.0, 'status': 'ok'},
 {'loss': -276.0, 'status': 'ok'},
 {'loss': 261.0, 'status': 'ok'},
 {'loss': -24.0, 'status': 'ok'},
 {'loss': -60.0, 'status': 'ok'},
 {'loss': 164.0, 'status': 'ok'},
 {'loss': -11.0, 'status': 'ok'},
 {'loss': 156.0, 'status': 'ok'},
 {'loss': 184.0, 'status': 'ok'},
 {'loss': -244.0, 'status': 'ok'},
 {'loss': -276.0, 'status': 'ok'},
 {'loss': 101.0, 'status': 'ok'},
 {'loss': 284.0, 'status': 'ok'},
 {'loss': -71.0, 'status': 'ok'},
 {'loss': 156.0, 'status': 'ok'},
 {'loss': -95.0, 'status': 'ok'},
 {'loss': 300.0, 'status': 'ok'},
 {'loss': -124.0, 'status': 'ok'},
 {'loss': -151.0, 'status': 'ok'},
 {'loss': -111.0, 'status': 'ok'},
 {'loss': -279.0, 'status': 'ok'},
 {'loss': -239.0, 'status': 'ok'},
 {'loss': -276.0, 'status': 'ok'},
 {'loss': -216.0, 'status': 'ok'},
 {'loss': -296.0, 'status': 'ok'},
 {'loss': -36.0, 'status': 'ok'},
 {'loss': -99.0, 'status': 'ok'},
 {'loss': -175.0, 'status': 'ok'},
 {'loss': 36.0, 'status': 'ok'},
 {'

In [14]:
# 탐색 과정 마다의 하이퍼 파라미터 값을 딕셔너리 형태로 저장
trials.vals

{'x': [np.float64(-9.0),
  np.float64(2.0),
  np.float64(-9.0),
  np.float64(6.0),
  np.float64(-10.0),
  np.float64(-8.0),
  np.float64(3.0),
  np.float64(4.0),
  np.float64(8.0),
  np.float64(-4.0),
  np.float64(-2.0),
  np.float64(-9.0),
  np.float64(8.0),
  np.float64(7.0),
  np.float64(-4.0),
  np.float64(-5.0),
  np.float64(-0.0),
  np.float64(-6.0),
  np.float64(7.0),
  np.float64(-7.0),
  np.float64(-1.0),
  np.float64(1.0),
  np.float64(-2.0),
  np.float64(2.0),
  np.float64(-2.0),
  np.float64(-2.0),
  np.float64(-1.0),
  np.float64(5.0),
  np.float64(-4.0),
  np.float64(-0.0),
  np.float64(-3.0),
  np.float64(2.0),
  np.float64(10.0),
  np.float64(-6.0),
  np.float64(1.0),
  np.float64(3.0),
  np.float64(1.0),
  np.float64(4.0),
  np.float64(10.0),
  np.float64(1.0),
  np.float64(4.0),
  np.float64(-3.0),
  np.float64(-10.0),
  np.float64(-8.0),
  np.float64(-1.0),
  np.float64(5.0),
  np.float64(-5.0),
  np.float64(3.0),
  np.float64(-3.0),
  np.float64(9.0),
  np.float64(6

### Optuna

In [15]:
import optuna

# 목적 함수
def objective(trial) :
    x = trial.suggest_uniform('x', -10, 10)
    y = trial.suggest_uniform('y', -15, 15)
    return x ** 2 + 20 * y

c:\Users\Playdata\miniconda3\envs\mlstudy_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# 스터디 생성
study = optuna.create_study(direction='minimize')

# 최적화 진행
study.optimize(objective, n_trials=500)

[I 2026-02-05 10:44:09,649] A new study created in memory with name: no-name-bd1d9a6e-0cf9-4bbf-b080-81e4030d63f7
C:\Users\Playdata\AppData\Local\Temp\ipykernel_26924\2463587924.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  x = trial.suggest_uniform('x', -10, 10)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_26924\2463587924.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  y = trial.suggest_uniform('y', -15, 15)
[I 2026-02-05 10:44:09,649] Trial 0 finished with value: 262.9952200182284 and parameters: {'x': 2.2637019023608786, 'y': 12.893543685773807}. Best is trial 0 with value: 262.9952200182284.
[I 2026-02-05 10:44:09,649] Trial 1 finished with value: 228.52533090111012 and parameters: {'x': 

In [17]:
# 최적의 값
study.best_value

-299.97627482853784

In [18]:
# 최적의 값을 내는 파라미터
study.best_params

{'x': -0.04489999493626656, 'y': -14.998914541904156}

In [19]:
import optuna.visualization as vis

vis.plot_param_importances(study).show()

In [20]:

vis.plot_optimization_history(study).show()

### [한번 해보기] hyperopt와 optuna를 활용한 XGBoost 하이퍼 파라미터 튜닝

In [22]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, random_state=42)

##### hyperopt 활용 최적화

In [24]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

# 1. 검색 공간 정의
search_space = {
    'n_estimators' : hp.quniform('n_estimators', 100, 500, 100)
    , 'max_depth' : hp.quniform('max_depth', 3, 10, 1)
    , 'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2)
    , 'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1)
}
# 2. 목적 함수 정의
def xgb_hyperopt_objective(space) :
    xgb_clf = XGBClassifier(
        n_estimators = int(space['n_estimators'])
        , max_depth = int(space['max_depth'])
        , learning_rate = space['learning_rate']
        , colsample_bytree = space['colsample_bytree']
    )
    mean_acc = cross_val_score(xgb_clf, X_train, y_train, cv=3).mean()
    return {
        'loss' : -1 * mean_acc
        , 'status' : hyperopt.STATUS_OK
    }
# 3. Trials() 생성
trials = Trials()
# 4. fmin() 최적화
hopt_best_xgb = fmin(
    fn=xgb_hyperopt_objective            # 목적 함수
    , space=search_space    # 검색 공간
    , algo=tpe.suggest      # 베이지안 최적화 적용
    , max_evals=50         # 반복 횟수 지정
    , trials=trials         # 탐생 과정 저장
)

100%|██████████| 50/50 [00:12<00:00,  3.88trial/s, best loss: -0.9741784037558686]


##### optuna 활용 최적화

In [26]:
# 1. 목적 함수 정의
def xgb_optuna_objective(trial) :
    params = {
        'n_estimators' : trial.suggest_int('n_estimators', 100, 500, 100)
        , 'max_depth' : trial.suggest_int('max_depth', 3, 10, 1)
        , 'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.2)
        , 'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1)
    }
    xgb_clf = XGBClassifier(**params)
    return cross_val_score(xgb_clf, X_train, y_train, cv=3).mean()
# 2. study 객체 생성
xgb_optuna_study = optuna.create_study(direction='maximize')

# 3. study 객체를 통한 최적화
xgb_optuna_study.optimize(xgb_optuna_objective, n_trials=50)

[I 2026-02-05 11:35:45,679] A new study created in memory with name: no-name-3f316403-e7dc-4ec6-a54a-a36fc39b9b07
C:\Users\Playdata\AppData\Local\Temp\ipykernel_26924\1124126558.py:4: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v3.5.0 for details.

C:\Users\Playdata\AppData\Local\Temp\ipykernel_26924\1124126558.py:5: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments i

In [27]:
print(f'HyperOpt 최적 파라미터 : {hopt_best_xgb}')
print(f'Optuna 최적 파라미터 : {xgb_optuna_study.best_params}')

HyperOpt 최적 파라미터 : {'colsample_bytree': np.float64(0.5075313199829103), 'learning_rate': np.float64(0.13471449173000014), 'max_depth': np.float64(8.0), 'n_estimators': np.float64(400.0)}
Optuna 최적 파라미터 : {'n_estimators': 400, 'max_depth': 8, 'learning_rate': 0.15205824285399638, 'colsample_bytree': 0.7430598695296741}


In [34]:
xgb_hopt = XGBClassifier(
    n_estimators = 400
    , max_depth = 8
    , learning_rate = 0.13
    , colsample_bytree = 0.5
)
xgb_optuna = XGBClassifier(
    n_estimators = 400
    , max_depth = 8
    , learning_rate = 0.15
    , colsample_bytree = 0.74
)

xgb_hopt.fit(X_train, y_train)
xgb_optuna.fit(X_train, y_train)

print(xgb_hopt.score(X_train, y_train), xgb_hopt.score(X_test, y_test))
print(xgb_optuna.score(X_train, y_train), xgb_optuna.score(X_test, y_test))

1.0 0.965034965034965
1.0 0.958041958041958
